In [ ]:
%load_ext autoreload
%autoreload 2
from pymongo import MongoClient
import sys
from pathlib import Path
from tqdm import tqdm
import json

sys.path.append(str(Path("..").resolve()))
from src import *

# Migrate MySQL to MongoDB
As the database schema has evolved significantly since the current MySQL database was created, we've first migrated a subset of the collections using [_MongoDB Relational Migrator_](https://www.mongodb.com/resources/solutions/use-cases/mysql-to-mongodb).

The database up to this point can be loaded from `backup/1_after_migration` using the following command.
```sh
mongorestore --host localhost:27017 --db omero_museum  "../backup/1_after_migration/"
```

In this notebook we are going to complete the migration using handwritten queries.

## Load post-migration data
The following cell reset the `omero_museum` database loading the `1_after_migration` snapshot.

In [ ]:
%%capture
# Start from scratch
MongoClient("mongodb://localhost:27017/").drop_database("omero_museum")
!mongorestore --host localhost:27017 --drop --db omero_museum  "../backup/1_after_migration/"

## Visualizing the migrated database

In [57]:
schema=json.load(open("schema.json"))
connector=MongoDBConnector("omero_museum")
db=connector.db

The collections of the omero_museum db are:
----------------------------------------
[activities]:
[artworks]: _id data descrizione id sala tipologia
[authors]: _id cognome data_nasc id luogo_nasc nome sesso
[departments]: _id nome piano posti_occ posti_tot stanza
[employees]: _id cellulare cognome curriculum data_nasc data_registrazione email id luogo_nasc nome sesso
[limited_events]:
[messages]:
[rooms]:
[suppliers]:
[surveys]: _id accompagnatori_visita data_compilazione motivazione_visita numero_visite ritorno tipologia_visita titolo_studi valutazione_esperienza valutazione_struttura valutazione_visita visitatore
[tickets]: _id costo data_stampa evento gruppo id
[visitors]: _id cellulare cognome email id name  tariffa tessera
[workshops]: _id costo_classe costo_persona durata nome tipologia


There are 3 kinds of problems that catch the eye:
1. Collections need to be **renamed**,
2. Some collections are **missing**.
3. Several fields have **changed** during the design.

### 1. Renaming fields

In [45]:
collections_renaming = {
    "biglietti": "tickets",
    "reparti": "departments",
    "dipendenti": "employees",
    "opere": "artworks",
    "clienti": "visitors",
    "laboratori": "workshops",
    "questionari": "surveys",
    "artisti": "authors",
}
for old_name, new_name in collections_renaming.items():
    if old_name not in connector.collections:
        continue
    db[old_name].rename(new_name)
    cprint("Renaming collection", f"red:{old_name} --> {new_name}")

Renaming collection biglietti --> tickets
Renaming collection reparti --> departments
Renaming collection dipendenti --> employees
Renaming collection opere --> artworks
Renaming collection clienti --> visitors
Renaming collection laboratori --> workshops
Renaming collection questionari --> surveys
Renaming collection artisti --> authors


### 2. Adding missing collections

In [46]:
collections_missing = [
    "rooms",
    "messages",
    "suppliers",
    "limited_events",
    "activities",
]
for name in collections_missing:
    db.create_collection(name)
    cprint("Creating collection", f"green:{name}")

Creating collection rooms
Creating collection messages
Creating collection suppliers
Creating collection limited_events
Creating collection activities


### 3. Renaming documents' entries

In [51]:
connector.stats()

The collections of the omero_museum db are:
----------------------------------------
[activities]:
[artworks]: _id data descrizione id sala tipologia
[authors]: _id cognome data_nasc id luogo_nasc nome sesso
[departments]: _id nome piano posti_occ posti_tot stanza
[employees]: _id cellulare cognome curriculum data_nasc data_registrazione email id luogo_nasc nome sesso
[limited_events]:
[messages]:
[rooms]:
[suppliers]:
[surveys]: _id accompagnatori_visita data_compilazione motivazione_visita numero_visite ritorno tipologia_visita titolo_studi valutazione_esperienza valutazione_struttura valutazione_visita visitatore
[tickets]: _id costo data_stampa evento gruppo id
[visitors]: _id cellulare cognome email id name  tariffa tessera
[workshops]: _id costo_classe costo_persona durata nome tipologia


In [ ]:
entries_renaming = {
    "artworks": {
        "data": "date",
        "descrizione": "description",
        "sala": "room",
        "tipologia": "type",
    },
    "authors": {
        "cognome": "surname",
        "data_nasc": "birth_date",
        "luogo_nasc": "home_town",
        "nome": "name",
        "sesso": "gender",
    },
    "departments": {
        "nome": "name",
        "piano": "floor",
        "posti_occ": "free_spots",
        "stanza": "room",
    },
    "employees": {
        "cellulare": "phone_number",
        "cognome": "surname",
        "curriculum": "curriculum",
        "data_nasc": "birth_date",
        "data_registrazione": "date_start",
        "email": "email",
        "luogo_nasc": "hometown",
        "nome": "name",
        "sesso": "gender",
    },
    "surveys": {
        "accompagnatori_visita": "accompanying_persons_visit",
        "data_compilazione": "date_of_compilation",
        "motivazione_visita": "reason_for_visit",
        "numero_visite": "number_of_visits",
        "ritorno": "return",
        "tipologia_visita": "type_of_visit",
        "titolo_studi": "title_of_studies",
        "valutazione_esperienza": "evaluation_of_experience",
        "valutazione_struttura": "evaluation_of_facility",
        "valutazione_visita": "evaluation_of_visit",
    },
    "tickets": {
        "costo": "price",
        "data_stampa": "date",
    },
    "visitors": {
        "cellulare": "cell phone number",
        "cognome": "surname",
        "email": "email",
        "name": "name",
        "tariffa": "fare",
    },
    "workshops": {
        "costo_classe": "price_class",
        "costo_persona": "price_person",
        "durata": "duration",
        "nome": "title",
        "tipologia": "type",
    },
}

In [ ]:
db.visitor.update_many({}, {"$unset": {"indirizzo": ""}})
db.visitor.update_many({}, {"$rename": {"nome": "name "}})

UpdateResult({'n': 0, 'nModified': 0, 'ok': 1.0, 'updatedExisting': False}, acknowledged=True)

## Setting right `_id` values